In [1]:
import os
import sys
import math
import time
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import Tensor
import audioread


from torchlibrosa.stft import Spectrogram, LogmelFilterBank # use 'magphase' to extract phase informatinon with magnitude
device = "cuda" if torch.cuda.is_available() else "cpu"

ModuleNotFoundError: No module named 'torchlibrosa'

In [24]:


# initialize and conv block
def init_layer(layer):
    """Initialize a Linear or Convolutional layer. """
    nn.init.xavier_uniform_(layer.weight)
 
    if hasattr(layer, 'bias'):
        if layer.bias is not None:
            layer.bias.data.fill_(0.)
            
    
def init_bn(bn):
    """Initialize a Batchnorm layer. """
    bn.bias.data.fill_(0.)
    bn.weight.data.fill_(1.)


def init_gru(rnn):
    """Initialize a GRU layer. """
    
    def _concat_init(tensor, init_funcs):
        (length, fan_out) = tensor.shape
        fan_in = length // len(init_funcs)
    
        for (i, init_func) in enumerate(init_funcs):
            init_func(tensor[i * fan_in : (i + 1) * fan_in, :])
        
    def _inner_uniform(tensor):
        fan_in = nn.init._calculate_correct_fan(tensor, 'fan_in')
        nn.init.uniform_(tensor, -math.sqrt(3 / fan_in), math.sqrt(3 / fan_in))
    
    for i in range(rnn.num_layers):
        _concat_init(
            getattr(rnn, 'weight_ih_l{}'.format(i)),
            [_inner_uniform, _inner_uniform, _inner_uniform]
        )
        torch.nn.init.constant_(getattr(rnn, 'bias_ih_l{}'.format(i)), 0)

        _concat_init(
            getattr(rnn, 'weight_hh_l{}'.format(i)),
            [_inner_uniform, _inner_uniform, nn.init.orthogonal_]
        )
        torch.nn.init.constant_(getattr(rnn, 'bias_hh_l{}'.format(i)), 0)
        
def init_ln(module):
    if isinstance(module, nn.Embedding):
        module.weight.data.normal_(mean=0.0, std=1.0)
        if module.padding_idx is not None:
            module.weight.data[module.padding_idx].zero_()
    elif isinstance(module, nn.LayerNorm):
        module.bias.data.zero_()
        module.weight.data.fill_(1.0)

class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x: Tensor) -> Tensor:
        """
        Args:
            x: Tensor, shape [seq_len, batch_size, embedding_dim]
        """
        x = x.transpose(0,1)
        print(x.shape)
        x = x + self.pe[:x.size(0)]
        x = x.transpose(0,1)
        return self.dropout(x)
    
class TransformerEncoder(nn.Module):
    def __init__(self, d_model=768, nhead=8):
        super(TransformerEncoder, self).__init__()
        
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead) #,batch_first=True) torch==1.8.0ではbatch_firstが使えない、、
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=4)
        
    def forward(self, inputs):
        inputs = inputs.transpose(0,1)
        output = self.transformer_encoder(inputs)
        output = output.transpose(0,1)
        return output


class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, momentum):
        
        super(ConvBlock, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=in_channels, 
                              out_channels=out_channels,
                              kernel_size=(3, 3), stride=(1, 1),
                              padding=(1, 1), bias=False)
                              
        self.conv2 = nn.Conv2d(in_channels=out_channels, 
                              out_channels=out_channels,
                              kernel_size=(3, 3), stride=(1, 1),
                              padding=(1, 1), bias=False)
                              
        self.bn1 = nn.BatchNorm2d(out_channels, momentum)
        self.bn2 = nn.BatchNorm2d(out_channels, momentum)

        self.init_weight()
        
    def init_weight(self):
        init_layer(self.conv1)
        init_layer(self.conv2)
        init_bn(self.bn1)
        init_bn(self.bn2)

        
    def forward(self, input, pool_size=(2, 2), pool_type='avg'):
        """
        Args:
          input: (batch_size, in_channels, time_steps, freq_bins)
        Outputs:
          output: (batch_size, out_channels, classes_num)
        """

        x = F.relu_(self.bn1(self.conv1(input)))
        #print(x)
        x = F.relu_(self.bn2(self.conv2(x)))
        
        if pool_type == 'avg':
            x = F.avg_pool2d(x, kernel_size=pool_size)
        
        return x


# In[4]:


# アコースティックモデル
class AcousticModelCRnn8Dropout(nn.Module):
    def __init__(self, classes_num, midfeat, momentum):
        super(AcousticModelCRnn8Dropout, self).__init__()

        self.conv_block1 = ConvBlock(in_channels=1, out_channels=48, momentum=momentum)
        self.conv_block2 = ConvBlock(in_channels=48, out_channels=64, momentum=momentum)
        self.conv_block3 = ConvBlock(in_channels=64, out_channels=96, momentum=momentum)
        self.conv_block4 = ConvBlock(in_channels=96, out_channels=128, momentum=momentum)

        self.fc5 = nn.Linear(midfeat, 768, bias=False)
        self.bn5 = nn.BatchNorm1d(768, momentum=momentum)

        self.gru = nn.GRU(input_size=768, hidden_size=256, num_layers=2, 
            bias=True, batch_first=True, dropout=0., bidirectional=True)

        self.fc = nn.Linear(512, classes_num, bias=True)
        
        self.init_weight()

    def init_weight(self):
        init_layer(self.fc5)
        init_bn(self.bn5)
        init_gru(self.gru)
        init_layer(self.fc)

    def forward(self, input):
        """
        Args:
          input: (batch_size, channels_num, time_steps, freq_bins)
        Outputs:
          output: (batch_size, time_steps, classes_num)
        """

        x = self.conv_block1(input, pool_size=(1, 2), pool_type='avg')
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv_block2(x, pool_size=(1, 2), pool_type='avg')
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv_block3(x, pool_size=(1, 2), pool_type='avg')
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv_block4(x, pool_size=(1, 2), pool_type='avg')
        x = F.dropout(x, p=0.2, training=self.training)

        x = x.transpose(1, 2).flatten(2)
        x = F.relu(self.bn5(self.fc5(x).transpose(1, 2)).transpose(1, 2))
        x = F.dropout(x, p=0.5, training=self.training, inplace=True)
        
        (x, _) = self.gru(x)
        x = F.dropout(x, p=0.5, training=self.training, inplace=False)
        output = torch.sigmoid(self.fc(x))
        return output 
    
# アコースティックモデル(Transformer バージョン)

# アコースティックモデル
class AcousticModelTransformer(nn.Module):
    def __init__(self, classes_num, midfeat, momentum):
        super(AcousticModelTransformer, self).__init__()

        self.conv_block1 = ConvBlock(in_channels=1, out_channels=48, momentum=momentum)
        self.conv_block2 = ConvBlock(in_channels=48, out_channels=64, momentum=momentum)
        self.conv_block3 = ConvBlock(in_channels=64, out_channels=96, momentum=momentum)
        self.conv_block4 = ConvBlock(in_channels=96, out_channels=128, momentum=momentum)

        self.fc5 = nn.Linear(midfeat, 768, bias=False)
        self.bn5 = nn.BatchNorm1d(768, momentum=momentum)
        
        self.ln1 = nn.LayerNorm(768)
        self.pe = PositionalEncoding(d_model=768, dropout=0.1)
        
        self.encoder_layer = TransformerEncoder()

        self.fc1 = nn.Linear(768, 512, bias=True)
        self.fc2 = nn.Linear(512, 256, bias=True)
        self.fc3 = nn.Linear(256, classes_num, bias=False)
        
        
        
        self.init_weight()

    def init_weight(self):
        init_layer(self.fc5)
        init_bn(self.bn5)
        init_layer(self.fc1)
        init_layer(self.fc2)
        init_layer(self.fc3)
        init_ln(self.ln1)

    def forward(self, input):
        """
        Args:
          input: (batch_size, channels_num, time_steps, freq_bins)
        Outputs:
          output: (batch_size, time_steps, classes_num)
        """

        x = self.conv_block1(input, pool_size=(1, 2), pool_type='avg')
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv_block2(x, pool_size=(1, 2), pool_type='avg')
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv_block3(x, pool_size=(1, 2), pool_type='avg')
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv_block4(x, pool_size=(1, 2), pool_type='avg')
        x = F.dropout(x, p=0.2, training=self.training) # [batch_size, 128, 1001, 14]
        
        x = x.transpose(1, 2).flatten(2)  # [batch_size, 1001, 1792]

        x = F.relu(self.fc5(x)) # [batch_size, 1001, 768]
        x = self.ln1(x)
        x = F.dropout(x, p=0.2, training=self.training, inplace=True)  # [batch_size, 1001, 768]
        
        print(x.shape)
        x = self.pe(x) # [batch_soze, 1001, 768]
        print(x.shape)
        
        x = self.encoder_layer(x)  # [batch_size, 1001, 768]
        x = F.dropout(x, p=0.5, training=self.training, inplace=False)
        x = self.fc1(x)
        x = self.fc2(x)
        
        output = torch.sigmoid(self.fc3(x))
        
        # この下にTransformerのEncoder部分(4層)を書く(to do)
        
        return output


In [27]:
inputs_batch = torch.ones(10, 1, 1001, 229)
NET = AcousticModelTransformer(classes_num=88, midfeat=1792, momentum=0.1)
outputs = NET(inputs_batch)

torch.Size([10, 1001, 768])
torch.Size([1001, 10, 768])
torch.Size([10, 1001, 768])


In [21]:
outputs.shape

torch.Size([10, 1001, 768])

In [17]:
torch.__version__

'1.8.0+cu111'

In [3]:
import torch

In [7]:
!pip install torch==1.8.0+cu111 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |################################| 1982.2 MB 10 kB/s              #######                   | 805.8 MB 3.4 MB/s eta 0:05:42 ########                   | 816.1 MB 23.7 MB/s eta 0:00:50#########                  | 923.7 MB 821 kB/s eta 0:21:28 ###########                | 1010.6 MB 941 kB/s eta 0:17:12               | 1043.4 MB 877 kB/s eta 0:17:50###########                | 1048.8 MB 877 kB/s eta 0:17:44############               | 1073.2 MB 907 kB/s eta 0:16:42############               | 1086.1 MB 973 kB/s eta 0:15:21#              | 1140.5 MB 910 kB/s eta 0:15:25##############             | 1207.2 MB 972 kB/s eta 0:13:18##############             | 1237.0 MB 911 kB/s eta 0:13:38###############            | 1256.0 MB 911 kB/s eta 0:13:17###            | 1264.1 MB 1.0 MB/s eta 0:11:32###############            | 1268.9 MB 1.0 MB/s eta 0:11:28###############            | 1294.6 MB 872 kB/s eta 0:13:09############

In [9]:
import audioread

In [10]:
audioread.__version__

'2.1.6'

In [11]:
!pip install audioread==3.0.0

     |################################| 377 kB 11.8 MB/s            
  Preparing metadata (setup.py) ... done
  Created wheel for audioread: filename=audioread-3.0.0-py3-none-any.whl size=23693 sha256=103938a40bc02d57106cbe6547f65d28d2281ea313e580d86fc620dfb9e7c75c
  Stored in directory: /root/.cache/pip/wheels/00/be/fc/a93c5810787b4f37cd2a5336f8291235efbf0da00bb04add66
Successfully built audioread
  Attempting uninstall: audioread
    Found existing installation: audioread 2.1.6
    Uninstalling audioread-2.1.6:
      Successfully uninstalled audioread-2.1.6
